In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

In [2]:
import random
import numpy as np
import skimpy
import skimpy_3d
from skimpy_blox import minecraft, colors
from skimpy import functional as F

In [3]:
%%time
level = minecraft.SkimpyMinecraftLevel.load("../data/sample_minecraft_world.pickle.gz")

Wall time: 1.91 s


In [4]:
%%time
#t = level.megatensor()
t = level.chunk_list[0].tensor

Wall time: 0 ns


In [5]:
for i in range(20):
    print(f"{i}=>{F.sum((t == i).to(int))}")

0=>4145
1=>14171
2=>250
3=>1091
4=>0
5=>0
6=>0
7=>256
8=>0
9=>20
10=>0
11=>62
12=>24
13=>60
14=>6
15=>95
16=>169
17=>0
18=>0
19=>0


In [6]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
for i in range(1, 256):
    r, g, b = colors.color_for_id(i)
    config[i] = skimpy_3d.ColorVoxel(r, g, b)

In [7]:
%%time
mesh = skimpy_3d.generate_mesh(config, t._tensor)

Wall time: 20 ms


In [8]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [9]:
%%time 

geometry = BufferGeometry(
    attributes={
        "position": BufferAttribute(
            np.array(mesh.positions, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "normal": BufferAttribute(
            np.array(mesh.normals, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "color": BufferAttribute(
            np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
        ),
        "index": BufferAttribute(
            np.array(mesh.triangles, dtype='uint32'),
            normalized=False,
        ),
    },
)

Wall time: 44 ms


In [10]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [11]:
js_mesh

Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(20080, 3), dtype=fl…

In [ ]:
camera_pos = level.xyz_to_skimpy_col(level.dense_dimensions())
camera_pos = (10, 10, 10)
camera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)
up = [0, 0, 0]
up[level.column_order[1]] = 1
camera.up = tuple(up)

In [ ]:
light_position = level.xyz_to_skimpy_col(tuple(e / 2 for e in level.dense_dimensions()))
point_light = PointLight(color='#ffffff', position=light_position)
global_light = AmbientLight(color='#333333')

#normals_helper = VertexNormalsHelper(js_mesh, size=0.5, color="#ffcc00")

scene = Scene(children=[js_mesh, camera, point_light, global_light])

In [ ]:
renderer = Renderer(
    camera=camera, 
    background="white", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)